# Workshop part 1 | Learn how to train a model
In this first part of the workshop, all preparation for training a model and the actual training are performed. 

The learning points are:
- How a prediction job works, and what the most important parameters mean; 
- What data is required;
- Experience with the train model pipeline;
- How the model gets automatically stored and loaded;
- How to get info on the trained model.

In [ ]:
# Import all required packages
import openstef
from openstef.data_classes.prediction_job import PredictionJobDataClass
from openstef.pipeline.train_model import train_model_pipeline

from IPython.display import IFrame
import pandas as pd 

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = 'plotly'

## Define the prediction job

OpenSTEF uses prediction jobs to define the properties of training and prediction. 

Exercise: define your own prediction job using the following parameters: 
- latitude = 53.0, longitude = 5.7
    - This is used to calculate the derived solar features (direct normal irradiance and the global tilted irradiance).*
    - Also used to retrieve weather data in openstef-dbc (database connector)
- horizon minutes = 2880
    - The horizon of the desired forecast in minutes. It entails how far into the future we want to predict. The value of 15 entails that at the moment of prediction, you predict 15 minutes into the future. So let's say you make a prediction at one o'clock, than the prediction is for 13.15 o'clock. 
- quantile: 10, 30, 50, 70 and 90 percent
    - This provides a confidence interval within OpenSTEF, based on the standard deviation.

Hint: look at the documentation [here](https://openstef.github.io/openstef/openstef.data_classes.html#module-openstef.data_classes.prediction_job)

*The code which calculated the direct normal irradiance and global tilted irradiance can be found in ``weather_features.py``, [here](https://github.com/OpenSTEF/openstef/blob/main/openstef/feature_engineering/weather_features.py).

In [ ]:
# Define properties of training/prediction. We call this a 'prediction_job'
pj = dict(id=287,
        model='xgb', 
        quantiles=[0.10,0.30,0.50,0.70,0.90],
        forecast_type="demand", 
        lat=53.0,
        lon=5.7,
        horizon_minutes=2880,
        resolution_minutes=15,
        name="workshop_exercise_1",
        save_train_forecasts=True,
       )

pj=PredictionJobDataClass(**pj)

## Prepare and analyse the input data
OpenSTEF requires a certain input format: a dataframe. 

Exercise: look at the table and plots below and answer try to answer the following questions: 
- What type of features do you see in the input data? 
- How much time is there between two data points? 
- Look at the plots for radiation and windspeed, do you see any paterns? 
    - Hint: do you see something happening to the load when there is a peak in either radiation or wind speed? Can you explain why? 
    - Note: in these plots we zoomed in on a random week, for visibility purposes. 

Hint: you can zoom in on the plots to see more details.

In [ ]:
input_data=pd.read_csv("../data/input_data_sun_heavy.csv", index_col=0, parse_dates=True)

In [ ]:
# Inspect all column names of the input data 
print(input_data.columns)

In [ ]:
pd.options.display.max_columns = None
display(input_data.head())

In [ ]:
# In the next section, the data will be split into training and testing data. The model should be only trained on the training part of the input data. Therefore, the input data should be split.
train_data=input_data.iloc[:-192,:] # Everything except the final 192 rows for training.
test_data=input_data.iloc[-192:,:] # Final 192 rows for testing.

In [ ]:
fig_load=input_data["load"].iloc[57:729].plot()
fig_load.update_layout(
    xaxis_title='Timestamp',
    yaxis_title="Load [MW]"
)
fig_load.show()

In [ ]:
fig_radiation=input_data["radiation"].iloc[57:729].plot()
fig_radiation.update_layout(
    xaxis_title='Timestamp',
    yaxis_title="rad"
)
fig_radiation.show()

In [ ]:
fig_windspeed=input_data["windspeed"].iloc[57:729].plot()
fig_windspeed.update_layout(
    xaxis_title='Timestamp',
    yaxis_title="Windspeed"
)
fig_windspeed.show()

## Training the model
After defining the prediction job and preparing the input data, the model can be trained. 

Exercise: 
- Using the prediction job and input data prepared above, train a model using the OpenSTEF pipelines. 
- How much time did it take to train the model?

Hint: find the correct pipeline in the list provided on the OpenSTEF [website](https://openstef.github.io/openstef/user_guides.html).

In [ ]:
train_data, validation_data, test_data=openstef.pipeline.train_model.train_model_pipeline(
    pj,
    train_data,
    check_old_model_age=False, 
    mlflow_tracking_uri="./mlflow_trained_models",
    artifact_folder="./mlflow_artifacts",
)

## Analyse the trained model
Now that the model has been trained, you can inspect the results. 

Exercise: answer the following questions: 
- Are all of the features in the feature importance plot in the input data? Why?
    - What are the most important features? 
- Which time horizon is more accurate? 
    - Hint: zoom in on the same day for both the Predictor0.25 and Predictor47.0 and examine them next to each other.
- Where is my trained model? 

The first two plots are the 'predictor in action' plots for the two time horizons (0.25 means fifteen minutes ahead, 47.0 means 47 hours ahead). In these plots you can see three different data outputs: train, validation and test. For each of these, you can see an '_actual' and '_predict'. This entails that for everyone of these data outputs, the measured value and the predicted value by OpenSTEF is plotted. Thus 'train_predict' is the prediction by OpenSTEF based on the train data.  

The last plot is the feature importance, this plot shows all of your input features (radiation, windspeed, lagged load, etc, etc,) and how much they influence the forecast. If a block is relatively large, this means the feature is relatively important for the forecast. Thus, large changes in the value of this feature results in a large difference in forecast. 

In [ ]:
# Inspect local files.
display(IFrame('./mlflow_artifacts/{}/Predictor0.25.html'.format(pj['id']), width=900, height=400))
display(IFrame('./mlflow_artifacts/{}/Predictor47.0.html'.format(pj['id']), width=800, height=400))
display(IFrame('./mlflow_artifacts/{}/weight_plot.html'.format(pj['id']), width=800, height=400))


## Visual Studio Code has difficulties with displaying htmls. If you are working with VSC and are not able to inspect the plots, uncomment the code below
## To open the plots in your browser.
#import webbrowser
#webbrowser.open('./mlflow_artifacts/{}/Predictor0.25.html'.format(pj['id']))
#webbrowser.open('./mlflow_artifacts/{}/Predictor47.0.html'.format(pj['id']))
#webbrowser.open('./mlflow_artifacts/{}/weight_plot.html'.format(pj['id']))